# National Park Analysis

In [20]:
#import libraries keys and setup data direcotry
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from datetime import date
import os
import sys
import random
from scipy.stats import linregress
from scipy.stats import normaltest
from citipy import citipy
import json


#Include api directory in path
if module_path not in sys.path:
    sys.path.append("api")

# Import API key
from config import g_key




## Import Park Attendance Data (Ana, Kevin-support)

In [ ]:
#Import in park atendnce excel/csv  

In [15]:
#Data cleanse.

## Import Operating Cost Data (Kevin)

In [11]:
#Import operating cost into data frame 

In [12]:
#Data cleansing

## Create Inflationary Function (Efrain, Mike-support)

In [18]:
# Create Inflationary function

## Geo Code Park Attendance Data Frame(Mike, Anna-support)

In [19]:
#Geocode work.  Utilizing existing Attendance 

## Merge Data add Custom fields (Efrain, Kevin, Ana)

In [ ]:
# Merge data frames add 

In [16]:
 #Create any custom fields needed to partition data for analysis

In [ ]:
#Duplicate all monetary fieldfs adjusting for inflation

## Create Heat Maps to Visualize Cost and Attendance (Kevin -  Mike support)

In [ ]:
## Heat Map By Operating Costs

In [17]:
##Heat Map by Attendance

## Scater/Line Plots Operating Costs versus Attendance (Ana and Efrain)  Be creative 

## Trend Analysis (Compare Productivity Year Over Year) Mike and Kevin